In [23]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [49]:
# Read data from excel file

xlsx = r"C:\Users\mfaraday01\Documents\Code\Aging Analysis\credit_debit_records.xlsx"
debit = pd.read_excel (xlsx, 
                     sheet_name="Debits"
                     )
credit = pd.read_excel (xlsx, 
                     sheet_name="Credit"
                     )

In [50]:
# Unpivot "debit df"

debit_unpiv = debit.melt(id_vars=["Zone", "Region", "District"], var_name="Date", value_name="Balance")

In [51]:
# Change datatypes

debit_unpiv[["Zone", "Region", "District"]] = debit_unpiv[["Zone", "Region", "District"]].astype(str)
debit_unpiv["Date"] = pd.to_datetime(debit_unpiv["Date"]).dt.date

In [52]:
# Add new calculated columns

debit_unpiv["Due date"] = debit_unpiv["Date"] + timedelta(days=7)
debit_unpiv["Overdue"] = (datetime.date(datetime.now()) - debit_unpiv["Due date"]).dt.days.astype('int16')

In [53]:
# Categorize overdue time to ages

conditions = [
               (debit_unpiv["Overdue"] <= 7),
               (debit_unpiv["Overdue"] > 7) & (debit_unpiv["Overdue"] <= 14),
               (debit_unpiv["Overdue"] > 14) & (debit_unpiv["Overdue"] <= 21),
               (debit_unpiv["Overdue"] > 21) & (debit_unpiv["Overdue"] <= 60),
               (debit_unpiv["Overdue"] > 60) & (debit_unpiv["Overdue"] <= 120),
               (debit_unpiv["Overdue"] > 120) & (debit_unpiv["Overdue"] <= 180),
               (debit_unpiv["Overdue"] > 180)               
               ]

choices = ["1-7 days", "8-15", "16-21", "22-60 days", "61-120 days", "121-180 days", "Above 180"]

debit_unpiv["Aging Status"] = np.select(conditions, choices)

In [ ]:
# Merge the two df

debit_credit = debit_unpiv.merge(credit, on="District")

In [ ]:
#Sort and re-index df

debit_credit = debit_credit.sort_values(by=["District", "Date"], ascending=True)
debit_credit = debit_credit.reset_index(drop=True)